In [196]:
import pandas as pd
import json
from collections import defaultdict

In [3]:
df = pd.read_csv('../data/crime_data.csv')

# EDA

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 944235 entries, 0 to 944234
Data columns (total 28 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   DR_NO           944235 non-null  int64  
 1   Date Rptd       944235 non-null  object 
 2   DATE OCC        944235 non-null  object 
 3   TIME OCC        944235 non-null  int64  
 4   AREA            944235 non-null  int64  
 5   AREA NAME       944235 non-null  object 
 6   Rpt Dist No     944235 non-null  int64  
 7   Part 1-2        944235 non-null  int64  
 8   Crm Cd          944235 non-null  int64  
 9   Crm Cd Desc     944235 non-null  object 
 10  Mocodes         811136 non-null  object 
 11  Vict Age        944235 non-null  int64  
 12  Vict Sex        817640 non-null  object 
 13  Vict Descent    817630 non-null  object 
 14  Premis Cd       944225 non-null  float64
 15  Premis Desc     943668 non-null  object 
 16  Weapon Used Cd  324477 non-null  float64
 17  Weapon Des

In [19]:
df.describe()

,DR_NO,TIME OCC,AREA,Rpt Dist No,Part 1-2,Crm Cd,Vict Age,Premis Cd,Weapon Used Cd,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LAT,LON
count,9.442350e+05,944235.000000,944235.00000,944235.000000,944235.000000,944235.000000,944235.000000,944225.000000,324477.000000,944224.000000,68258.000000,2281.000000,64.00000,944235.000000,944235.000000
mean,2.188777e+08,1338.047956,10.72113,1118.554020,1.409594,500.810848,29.499314,306.554488,363.710143,500.556373,958.083639,984.021920,991.21875,33.992364,-118.070963
std,1.246230e+07,652.446103,6.09928,609.969102,0.491759,207.283501,21.882246,217.561137,123.570915,207.076911,110.336391,51.798407,27.06985,1.670190,5.789397
min,8.170000e+02,1.000000,1.00000,101.000000,1.000000,110.000000,-4.000000,101.000000,101.000000,110.000000,210.000000,310.000000,821.00000,0.000000,-118.667600
25%,2.105081e+08,900.000000,6.00000,622.000000,1.000000,331.000000,0.000000,101.000000,311.000000,331.000000,998.000000,998.000000,998.00000,34.014500,-118.430700
50%,2.206203e+08,1419.000000,11.00000,1142.000000,1.000000,442.000000,30.000000,203.000000,400.000000,442.000000,998.000000,998.000000,998.00000,34.058900,-118.322500
75%,2.307061e+08,1900.000000,16.00000,1619.000000,2.000000,626.000000,45.000000,501.000000,400.000000,626.000000,998.000000,998.000000,998.00000,34.164900,-118.274000
max,2.499138e+08,2359.000000,21.00000,2199.000000,2.000000,956.000000,120.000000,976.000000,516.000000,956.000000,999.000000,999.000000,999.00000,34.334300,0.000000


In [33]:
len(df['Vict Descent'].unique())

21

In [14]:
df['DATE OCC'].values

array(['03/01/2020 12:00:00 AM', '02/08/2020 12:00:00 AM',
       '11/04/2020 12:00:00 AM', ..., '01/15/2024 12:00:00 AM',
       '04/24/2024 12:00:00 AM', '04/29/2024 12:00:00 AM'], dtype=object)

In [26]:
len(df[[a.find("/2023") != -1 for a in df['DATE OCC'].values]])

231642

In [208]:
df_2023 = df[[a.find("/2023") != -1 for a in df['DATE OCC'].values]]

In [209]:
df_2023['Rpt Dist No']

644378     668
644379     191
644380    1549
644381     192
644382     663
          ... 
876015    1602
876016    1798
876017     154
876018     914
876019     395
Name: Rpt Dist No, Length: 231642, dtype: int64

# Preprocessing

In [211]:
df_crime_cd_mapping = df[['Crm Cd', 'Crm Cd Desc']]
df_crime_cd_mapping.head()

,Crm Cd,Crm Cd Desc
0,510,VEHICLE - STOLEN
1,330,BURGLARY FROM VEHICLE
2,480,BIKE - STOLEN
3,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER)
4,354,THEFT OF IDENTITY


In [212]:
list(df_crime_cd_mapping['Crm Cd Desc'].unique())

['VEHICLE - STOLEN',
 'BURGLARY FROM VEHICLE',
 'BIKE - STOLEN',
 'SHOPLIFTING-GRAND THEFT ($950.01 & OVER)',
 'THEFT OF IDENTITY',
 'BATTERY - SIMPLE ASSAULT',
 'SODOMY/SEXUAL CONTACT B/W PENIS OF ONE PERS TO ANUS OTH',
 'CRM AGNST CHLD (13 OR UNDER) (14-15 & SUSP 10 YRS OLDER)',
 'SEX,UNLAWFUL(INC MUTUAL CONSENT, PENETRATION W/ FRGN OBJ',
 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT',
 'LETTERS, LEWD  -  TELEPHONE CALLS, LEWD',
 'THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LIVESTK,PROD',
 'CRIMINAL THREATS - NO WEAPON DISPLAYED',
 'EMBEZZLEMENT, GRAND THEFT ($950.01 & OVER)',
 'THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)',
 'CHILD ANNOYING (17YRS & UNDER)',
 'BURGLARY',
 'CONTEMPT OF COURT',
 'THEFT PLAIN - PETTY ($950 & UNDER)',
 'INTIMATE PARTNER - SIMPLE ASSAULT',
 'LEWD CONDUCT',
 'THEFT PLAIN - ATTEMPT',
 'THEFT FROM MOTOR VEHICLE - GRAND ($950.01 AND OVER)',
 'ROBBERY',
 'BUNCO, GRAND THEFT',
 'BATTERY WITH SEXUAL CONTACT',
 'INTIMATE PARTNER - AGGRAVATED ASSAULT',
 'OR

In [151]:
df_related = df[['Rpt Dist No', 'Crm Cd Desc', 'Vict Age', 'Vict Sex', 'Vict Descent']]
df_related.head()

,Rpt Dist No,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent
0,784,VEHICLE - STOLEN,0,M,O
1,182,BURGLARY FROM VEHICLE,47,M,O
2,356,BIKE - STOLEN,19,X,X
3,964,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),19,M,O
4,666,THEFT OF IDENTITY,28,M,H


In [152]:
ucr_categories_dict = {
    'homicide': [
        'CRIMINAL HOMICIDE',
        'LYNCHING - ATTEMPTED',
        'MANSLAUGHTER, NEGLIGENT',
        'LYNCHING'
    ],
    'rape': [
        'SODOMY/SEXUAL CONTACT B/W PENIS OF ONE PERS TO ANUS OTH',
        'SEX, UNLAWFUL(INC MUTUAL CONSENT, PENETRATION W/ FRGN OBJ)',
        'ORAL COPULATION', 
        'RAPE, FORCIBLE', 
        'RAPE, ATTEMPTED',
        'SEXUAL PENETRATION W/FOREIGN OBJECT',
        'SEX OFFENDER REGISTRANT OUT OF COMPLIANCE'
    ],
    'robbery': [
        'ROBBERY', 
        'ATTEMPTED ROBBERY', 
        'PURSE SNATCHING', 
        'PURSE SNATCHING - ATTEMPT'
    ],
    'aggravated_assault': [
        'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT',
        'ASSAULT WITH DEADLY WEAPON ON POLICE OFFICER',
        'CHILD ABUSE (PHYSICAL) - AGGRAVATED ASSAULT',
        'INTIMATE PARTNER - AGGRAVATED ASSAULT',
        'BATTERY WITH SEXUAL CONTACT', 
        'THREATENING PHONE CALLS/LETTERS',
        'CRIMINAL THREATS - NO WEAPON DISPLAYED', 
        'OTHER ASSAULT',
        'CRUELTY TO ANIMALS',
        'THROWING OBJECT AT MOVING VEHICLE'
    ],
    'kidnapping': ['KIDNAPPING', 'KIDNAPPING - GRAND ATTEMPT'],
    'burglary': [
        'BURGLARY', 
        'BURGLARY FROM VEHICLE', 
        'BURGLARY, ATTEMPTED',
        'BURGLARY FROM VEHICLE, ATTEMPTED'
    ],
    'contributing': ['CONTRIBUTING'],
    'larceny_theft': [
        'THEFT PLAIN - PETTY ($950 & UNDER)', 
        'THEFT PLAIN - ATTEMPT',
        'THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)',
        'THEFT FROM MOTOR VEHICLE - GRAND ($950.01 AND OVER)',
        'THEFT FROM MOTOR VEHICLE - ATTEMPT', 
        'VEHICLE - STOLEN',
        'VEHICLE - ATTEMPT STOLEN', 
        'BIKE - STOLEN', 
        'BIKE - ATTEMPTED STOLEN',
        'SHOPLIFTING-GRAND THEFT ($950.01 & OVER)', 
        'SHOPLIFTING - PETTY THEFT ($950 & UNDER)',
        'EMBEZZLEMENT, GRAND THEFT ($950.01 & OVER)', 
        'EMBEZZLEMENT, PETTY THEFT ($950 & UNDER)',
        'THEFT OF IDENTITY', 
        'THEFT, PERSON', 
        'THEFT FROM PERSON - ATTEMPT',
        'THEFT, COIN MACHINE - PETTY ($950 & UNDER)', 
        'THEFT, COIN MACHINE - ATTEMPT',
        'THEFT, COIN MACHINE - GRAND ($950.01 & OVER)', 
        'BUNCO, GRAND THEFT',
        'BUNCO, PETTY THEFT', 
        'BUNCO, ATTEMPT',
        'DEFRAUDING INNKEEPER/THEFT OF SERVICES, $950 & UNDER',
        'DEFRAUDING INNKEEPER/THEFT OF SERVICES, OVER $950.01',
        'DISHONEST EMPLOYEE - GRAND THEFT', 
        'DISHONEST EMPLOYEE - PETTY THEFT',
        'DISHONEST EMPLOYEE ATTEMPTED THEFT', 
        'DOCUMENT FORGERY / STOLEN FELONY',
        'DOCUMENT WORTHLESS ($200.01 & OVER)', 
        'DOCUMENT WORTHLESS ($200 & UNDER)',
        'CREDIT CARDS, FRAUD USE ($950.01 & OVER)', 
        'CREDIT CARDS, FRAUD USE ($950 & UNDER',
        'COUNTERFEIT', 
        'EXTORTION', 
        'UNAUTHORIZED COMPUTER ACCESS', 
        'CONSPIRACY',
        'BRIBERY', 
        'GRAND THEFT / AUTO REPAIR', 
        'GRAND THEFT / INSURANCE FRAUD',
        'FAILURE TO YIELD',
        'SHOPLIFTING - ATTEMPT', 
        'ILLEGAL DUMPING', 
        'OTHER MISCELLANEOUS CRIME',
        'PETTY THEFT - AUTO REPAIR',
        'TILL TAP - GRAND THEFT ($950.01 & OVER)',
        'TILL TAP - PETTY ($950 & UNDER)',
        'PICKPOCKET',
        'PICKPOCKET, ATTEMPT',
        'TRESPASSING'
    ],
    'grand_theft_auto': [
        'VEHICLE - STOLEN', 
        'VEHICLE - ATTEMPT STOLEN',
        'VEHICLE, STOLEN - OTHER (MOTORIZED SCOOTERS, BIKES, ETC)',
        'DRIVING WITHOUT OWNER CONSENT (DWOC)', 
        'BOAT - STOLEN',
        'THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LIVESTK,PROD'
    ],
    'arson': [
        'ARSON'
    ],
    'forgery': [
        'DOCUMENT FORGERY / STOLEN FELONY',
        'DOCUMENT WORTHLESS ($200.01 & OVER)',
        'DOCUMENT WORTHLESS ($200 & UNDER)',
        'COUNTERFEIT'
    ],
    'fraud': [
        'CREDIT CARDS, FRAUD USE ($950.01 & OVER)',
        'CREDIT CARDS, FRAUD USE ($950 & UNDER',
        'EXTORTION',
        'UNAUTHORIZED COMPUTER ACCESS',
        'CONSPIRACY',
        'BRIBERY',
        'DEFRAUDING INNKEEPER/THEFT OF SERVICES, $950 & UNDER',
        'DEFRAUDING INNKEEPER/THEFT OF SERVICES, OVER $950.01',
        'THEFT OF IDENTITY'
    ],
    'sex_offenses_felonies': [
        'SODOMY/SEXUAL CONTACT B/W PENIS OF ONE PERS TO ANUS OTH',
        'SEX,UNLAWFUL(INC MUTUAL CONSENT, PENETRATION W/ FRGN OBJ',
        'ORAL COPULATION', 
        'RAPE, FORCIBLE', 
        'RAPE, ATTEMPTED',
        'LEWD/LASCIVIOUS ACTS WITH CHILD',
        'CHILD PORNOGRAPHY',
        'HUMAN TRAFFICKING - INVOLUNTARY SERVITUDE',
        'HUMAN TRAFFICKING - COMMERCIAL SEX ACTS',
        'PIMPING',
        'PANDERING'
    ],
    'sex_offenses_misdemeanors': [
        'INDECENT EXPOSURE',
        'PEEPING TOM',
        'LEWD CONDUCT',
        'CHILD ANNOYING (17YRS & UNDER)',
        'BEASTIALITY, CRIME AGAINST NATURE SEXUAL ASSLT WITH ANIM',
        'LETTERS, LEWD  -  TELEPHONE CALLS, LEWD',

    ],
    'non_aggravated_assaults': [
        'BATTERY - SIMPLE ASSAULT',
        'INTIMATE PARTNER - SIMPLE ASSAULT',
        'CHILD ABUSE (PHYSICAL) - SIMPLE ASSAULT',
        'BATTERY POLICE (SIMPLE)',
        'FALSE IMPRISONMENT',
        'BATTERY WITH SEXUAL CONTACT',
        'BATTERY ON A FIREFIGHTER',
        'PROWLER',
        'STALKING'
    ],
    'weapon_laws': [
        'WEAPONS POSSESSION/BOMBING',
        'FIREARMS RESTRAINING ORDER (FIREARMS RO)',
        'BRANDISH WEAPON',
        'DISCHARGE FIREARMS/SHOTS FIRED',
        'SHOTS FIRED AT INHABITED DWELLING',
        'SHOTS FIRED AT MOVING VEHICLE, TRAIN OR AIRCRAFT',
        'FIREARMS EMERGENCY PROTECTIVE ORDER (FIREARMS EPO)'
    ],
    'offenses_against_family': [
        'CHILD ABANDONMENT',
        'CHILD NEGLECT (SEE 300 W.I.C.)',
        'CHILD STEALING',
        'CRM AGNST CHLD (13 OR UNDER) (14-15 & SUSP 10 YRS OLDER)',
        'CONTEMPT OF COURT',
        'VIOLATION OF RESTRAINING ORDER',
        'VIOLATION OF TEMPORARY RESTRAINING ORDER',
        'VIOLATION OF COURT ORDER',
        'BIGAMY',
        'INCEST (SEXUAL ACTS BETWEEN BLOOD RELATIVES)'
    ],
    'narcotics': [
        'DRUGS, TO A MINOR'
    ],
    'drunk_alcohol_drugs': [
        'DRUNK ROLL'
    ],
    'disorderly_conduct': [
        'DISTURBING THE PEACE',
        'INCITING A RIOT',
        'DISRUPT SCHOOL',
        'FAILURE TO DISPERSE',
        'BLOCKING DOOR INDUCTION CENTER',
        'BOMB SCARE',
        'RESISTING ARREST'
    ],
    'drunk_driving': [
        'RECKLESS DRIVING',
        'FAILURE TO YIELD'
    ],
    'vehicle_boating_laws': [
        'DRIVING WITHOUT OWNER CONSENT (DWOC)',
        'REPLICA FIREARMS(SALE,DISPLAY,MANUFACTURE OR DISTRIBUTE)'
    ],
    'vandalism': [
        'VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)',
        'VANDALISM - MISDEAMEANOR ($399 OR UNDER)',
        'TELEPHONE PROPERTY - DAMAGE',
        'TRAIN WRECKING'
    ],
    'receiving_stolen_property': [
        'RECEIVING STOLEN PROPERTY'
    ],
    'federal_offenses_wo_money': [
        'FALSE POLICE REPORT',
        'CONSPIRACY',
        'EXTORTION',
        'ILLEGAL DUMPING',
        'INCITING A RIOT',
        'BLOCKING DOOR INDUCTION CENTER'
    ]
}

In [153]:
len(ucr_categories_dict.keys())

25

In [154]:
ucr_cat_score_dict = {
    'homicide': 5,
    'rape': 5,
    'kidnapping': 5,
    'sex_offenses_felonies': 5,
    'aggravated_assault': 4,
    'sex_offenses_misdemeanors': 4,
    'arson': 4,
    'non_aggravated_assaults': 4,
    'weapon_laws': 3,
    'grand_theft_auto': 3,
    'narcotics': 2,
    'drunk_alcohol_drugs': 2,
    'robbery': 2,
    'burglary': 2,
    'larceny_theft': 2,
    'offenses_against_family': 2,
    'drunk_driving': 2,
    'federal_offenses_wo_money': 2,
    'vandalism': 1,
    'disorderly_conduct': 1,
    'forgery': 1,
    'fraud': 1,
    'vehicle_boating_laws': 1,
    'receiving_stolen_property': 1,
    'contributing': 0,
}
len(ucr_cat_score_dict.keys())

25

In [155]:
ucr_categories_dict.items()

dict_items([('homicide', ['CRIMINAL HOMICIDE', 'LYNCHING - ATTEMPTED', 'MANSLAUGHTER, NEGLIGENT', 'LYNCHING']), ('rape', ['SODOMY/SEXUAL CONTACT B/W PENIS OF ONE PERS TO ANUS OTH', 'SEX, UNLAWFUL(INC MUTUAL CONSENT, PENETRATION W/ FRGN OBJ)', 'ORAL COPULATION', 'RAPE, FORCIBLE', 'RAPE, ATTEMPTED', 'SEXUAL PENETRATION W/FOREIGN OBJECT', 'SEX OFFENDER REGISTRANT OUT OF COMPLIANCE']), ('robbery', ['ROBBERY', 'ATTEMPTED ROBBERY', 'PURSE SNATCHING', 'PURSE SNATCHING - ATTEMPT']), ('aggravated_assault', ['ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT', 'ASSAULT WITH DEADLY WEAPON ON POLICE OFFICER', 'CHILD ABUSE (PHYSICAL) - AGGRAVATED ASSAULT', 'INTIMATE PARTNER - AGGRAVATED ASSAULT', 'BATTERY WITH SEXUAL CONTACT', 'THREATENING PHONE CALLS/LETTERS', 'CRIMINAL THREATS - NO WEAPON DISPLAYED', 'OTHER ASSAULT', 'CRUELTY TO ANIMALS', 'THROWING OBJECT AT MOVING VEHICLE']), ('kidnapping', ['KIDNAPPING', 'KIDNAPPING - GRAND ATTEMPT']), ('burglary', ['BURGLARY', 'BURGLARY FROM VEHICLE', 'BURGLARY, 

In [156]:
cats_score = {}
for k, v in ucr_categories_dict.items():
    for c in v:
        cats_score[c] = ucr_cat_score_dict[k]
cats_score

{'CRIMINAL HOMICIDE': 5,
 'LYNCHING - ATTEMPTED': 5,
 'MANSLAUGHTER, NEGLIGENT': 5,
 'LYNCHING': 5,
 'SODOMY/SEXUAL CONTACT B/W PENIS OF ONE PERS TO ANUS OTH': 5,
 'SEX, UNLAWFUL(INC MUTUAL CONSENT, PENETRATION W/ FRGN OBJ)': 5,
 'ORAL COPULATION': 5,
 'RAPE, FORCIBLE': 5,
 'RAPE, ATTEMPTED': 5,
 'SEXUAL PENETRATION W/FOREIGN OBJECT': 5,
 'SEX OFFENDER REGISTRANT OUT OF COMPLIANCE': 5,
 'ROBBERY': 2,
 'ATTEMPTED ROBBERY': 2,
 'PURSE SNATCHING': 2,
 'PURSE SNATCHING - ATTEMPT': 2,
 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT': 4,
 'ASSAULT WITH DEADLY WEAPON ON POLICE OFFICER': 4,
 'CHILD ABUSE (PHYSICAL) - AGGRAVATED ASSAULT': 4,
 'INTIMATE PARTNER - AGGRAVATED ASSAULT': 4,
 'BATTERY WITH SEXUAL CONTACT': 4,
 'THREATENING PHONE CALLS/LETTERS': 4,
 'CRIMINAL THREATS - NO WEAPON DISPLAYED': 4,
 'OTHER ASSAULT': 4,
 'CRUELTY TO ANIMALS': 4,
 'THROWING OBJECT AT MOVING VEHICLE': 4,
 'KIDNAPPING': 5,
 'KIDNAPPING - GRAND ATTEMPT': 5,
 'BURGLARY': 2,
 'BURGLARY FROM VEHICLE': 2,
 'BURGLA

In [158]:
vals = []
for a in ucr_categories_dict.values():
    for b in a:
        vals.append(b)
vals

['CRIMINAL HOMICIDE',
 'LYNCHING - ATTEMPTED',
 'MANSLAUGHTER, NEGLIGENT',
 'LYNCHING',
 'SODOMY/SEXUAL CONTACT B/W PENIS OF ONE PERS TO ANUS OTH',
 'SEX, UNLAWFUL(INC MUTUAL CONSENT, PENETRATION W/ FRGN OBJ)',
 'ORAL COPULATION',
 'RAPE, FORCIBLE',
 'RAPE, ATTEMPTED',
 'SEXUAL PENETRATION W/FOREIGN OBJECT',
 'SEX OFFENDER REGISTRANT OUT OF COMPLIANCE',
 'ROBBERY',
 'ATTEMPTED ROBBERY',
 'PURSE SNATCHING',
 'PURSE SNATCHING - ATTEMPT',
 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT',
 'ASSAULT WITH DEADLY WEAPON ON POLICE OFFICER',
 'CHILD ABUSE (PHYSICAL) - AGGRAVATED ASSAULT',
 'INTIMATE PARTNER - AGGRAVATED ASSAULT',
 'BATTERY WITH SEXUAL CONTACT',
 'THREATENING PHONE CALLS/LETTERS',
 'CRIMINAL THREATS - NO WEAPON DISPLAYED',
 'OTHER ASSAULT',
 'CRUELTY TO ANIMALS',
 'THROWING OBJECT AT MOVING VEHICLE',
 'KIDNAPPING',
 'KIDNAPPING - GRAND ATTEMPT',
 'BURGLARY',
 'BURGLARY FROM VEHICLE',
 'BURGLARY, ATTEMPTED',
 'BURGLARY FROM VEHICLE, ATTEMPTED',
 'CONTRIBUTING',
 'THEFT PLAIN - P

In [159]:
df_related['Rpt Dist No'].value_counts()

Rpt Dist No
162     4976
645     4684
1494    4519
182     4480
646     4156
        ... 
704        1
1784       1
225        1
1053       1
709        1
Name: count, Length: 1209, dtype: int64

In [160]:
df_related[['Crm Cd Desc']] = df_related[['Crm Cd Desc']].map(lambda x: cats_score[x]).fillna(0)

/var/folders/bm/2p55x4d9111982mgcnj9dcm80000gn/T/ipykernel_17634/75661267.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_related[['Crm Cd Desc']] = df_related[['Crm Cd Desc']].map(lambda x: cats_score[x]).fillna(0)


In [136]:
df_related[['Crm Cd Desc']]

,Crm Cd Desc
0,3
1,2
2,2
3,2
4,1
...,...
944230,2
944231,4
944232,1
944233,4


In [161]:
districts = list()
for x in df_related['Rpt Dist No'].unique():
    districts.append(df_related[df_related['Rpt Dist No'] == x])
districts

[        Rpt Dist No  Crm Cd Desc  Vict Age Vict Sex Vict Descent
 0               784            3         0        M            O
 1803            784            1        31        F            B
 2090            784            2        64        M            O
 2659            784            3         0      NaN          NaN
 3241            784            4        67        F            B
 ...             ...          ...       ...      ...          ...
 937069          784            4        27        F            B
 938215          784            2        41        F            B
 941331          784            4        51        M            H
 942292          784            2        64        F            H
 942745          784            2         0      NaN          NaN
 
 [664 rows x 5 columns],
         Rpt Dist No  Crm Cd Desc  Vict Age Vict Sex Vict Descent
 1               182            2        47        M            O
 6               182            1        25      

In [162]:
districts[0][['Crm Cd Desc']]

,Crm Cd Desc
0,3
1803,1
2090,2
2659,3
3241,4
...,...
937069,4
938215,2
941331,4
942292,2


In [166]:
districts[0][['Crm Cd Desc']].sum()

Crm Cd Desc    1636
dtype: int64

In [174]:
{'A': 'asian', 'B': 'black', 'C': 'asian', 'D': 'asian', 'F': 'pacific', 'G':'pacific', 'H':'hispanic', 'I':'other', 'J':'asian', 'K':'asian', 'L':'asian', 'O':'other', 'P':'pacific', 'S':'pacific', 'U':'pacific', 'V':'asian', 'W':'white', 'X':'other', 'Z':'indian'}

{'A': 'asian',
 'B': 'black',
 'C': 'asian',
 'D': 'asian',
 'F': 'pacific',
 'G': 'pacific',
 'H': 'hispanic',
 'I': 'other',
 'J': 'asian',
 'K': 'asian',
 'L': 'asian',
 'O': 'other',
 'P': 'pacific',
 'S': 'pacific',
 'U': 'pacific',
 'V': 'asian',
 'W': 'white',
 'X': 'other',
 'Z': 'indian'}

In [205]:
db_data = defaultdict(dict)
tot_severity = int(df_related[['Crm Cd Desc']].sum())

for district in districts:
    dist_num = str(district.iloc[0]['Rpt Dist No'])
    district_sev = int(district[['Crm Cd Desc']].sum())
    woman, man, otherSex = 0, 0, 0
    asian, black, hispanic, white, pacific, indian, otherDescent = 0, 0, 0, 0, 0, 0, 0
    minor, earlyTwenty, lateTwenty, thirties, midlife, eldery = 0, 0, 0, 0, 0, 0

    for i in range(district.shape[0]):
        record = district.iloc[i]
        record_sev = record['Crm Cd Desc']

        match (record['Vict Sex']):
            case 'F':
                woman += record_sev
            case 'M':
                man += record_sev
            case _:
                otherSex += record_sev
        
        match (record['Vict Descent']):
            case 'A' | 'C' | 'D' | 'J' | 'K' | 'L' | 'V':
                asian += record_sev
            case 'B':
                black += record_sev
            case 'H':
                hispanic += record_sev
            case 'W':
                white += record_sev
            case 'F' | 'G' | 'P' | 'S' | 'U':
                pacific += record_sev
            case 'Z':
                indian += record_sev
            case _:
                otherDescent += record_sev

        if (record['Vict Age'] <= 0):
            pass
        elif (record['Vict Age'] < 18):
            minor += record_sev
        elif (record['Vict Age'] < 25):
            earlyTwenty += record_sev
        elif (record['Vict Age'] < 30):
            lateTwenty += record_sev
        elif (record['Vict Age'] < 40):
            thirties += record_sev
        elif (record['Vict Age'] < 65):
            midlife += record_sev
        else:
            eldery += record_sev

    db_data[dist_num] = {
        'default_score': district_sev / tot_severity / 2,
        'woman_score': woman / district_sev,
        'man_score': man / district_sev,
        'otherSex': otherSex / district_sev,
        'asian_score': asian / district_sev,
        'black_score': black / district_sev,
        'hispanic_score': hispanic / district_sev,
        'white_score': white / district_sev,
        'pacific_score': pacific / district_sev,
        'indian_score': indian / district_sev,
        'otherDescent_score': otherDescent / district_sev,
        'minor_score': minor / district_sev,
        'earlyTwenty_score': earlyTwenty / district_sev,
        'lateTwenty_score': lateTwenty / district_sev,
        'thirties_score': thirties / district_sev,
        'midlife_score': midlife / district_sev,
        'eldery_score': eldery / district_sev
    }
db_data

/var/folders/bm/2p55x4d9111982mgcnj9dcm80000gn/T/ipykernel_17634/2487849054.py:2: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  tot_severity = int(df_related[['Crm Cd Desc']].sum())
/var/folders/bm/2p55x4d9111982mgcnj9dcm80000gn/T/ipykernel_17634/2487849054.py:6: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  district_sev = int(district[['Crm Cd Desc']].sum())
/var/folders/bm/2p55x4d9111982mgcnj9dcm80000gn/T/ipykernel_17634/2487849054.py:6: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  district_sev = int(district[['Crm Cd Desc']].sum())
/var/folders/bm/2p55x4d9111982mgcnj9dcm80000gn/T/ipykernel_17634/2487849054.py:6: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeEr

defaultdict(dict,
            {'784': {'default_score': 0.00033867386410731907,
              'woman_score': 0.3808068459657702,
              'man_score': 0.3918092909535452,
              'otherSex': 0.2273838630806846,
              'asian_score': 0.023227383863080684,
              'black_score': 0.2561124694376528,
              'hispanic_score': 0.3245721271393643,
              'white_score': 0.09718826405867971,
              'pacific_score': 0.0024449877750611247,
              'indian_score': 0.0,
              'otherDescent_score': 0.29645476772616136,
              'minor_score': 0.023838630806845965,
              'earlyTwenty_score': 0.08007334963325183,
              'lateTwenty_score': 0.09229828850855745,
              'thirties_score': 0.18887530562347188,
              'midlife_score': 0.3215158924205379,
              'eldery_score': 0.04767726161369193},
             '182': {'default_score': 0.0022922589836554667,
              'woman_score': 0.35247900298022217,
 

In [206]:
json.dump(db_data, open('../data/db_data.json', 'w'))